In [11]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [22]:
data = pd.read_csv("C:/Users/hyeseung/Desktop/4-2/ML/1202/건대입구역_최종데이터.csv")
data

,Transport_Date,Station_Name,Boarding_Alighting,Passenger_Count,Typhoon,Heatwave,Temperature,Precipitation,Relative_Humidity,Commute,...,일월날짜,승하차,cue,습도,적설,파업,IsHoliday,이전 승하차수,일주일 전 승하차 수,잠실 스포츠
0,2022.1.1,건대입구,승차,362,0,0,-10.1,0.0,0,1,...,1,1,0,53,0.0,0,1,362,362,0
1,2022.1.1,건대입구,하차,180,0,0,-10.1,0.0,0,1,...,1,0,0,53,0.0,0,1,362,180,0
2,2022.1.1,건대입구,승차,1606,0,0,-0.9,0.0,0,0,...,1,1,0,38,0.0,0,1,180,1606,0
3,2022.1.1,건대입구,하차,2758,0,0,-0.9,0.0,0,0,...,1,0,0,38,0.0,0,1,1606,2758,0
4,2022.1.2,건대입구,승차,346,0,0,-2.2,0.0,0,1,...,2,1,0,66,0.1,0,0,2758,346,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,2023.8.30,건대입구,하차,3619,0,0,23.1,0.9,0,0,...,607,0,0,54,0.0,0,0,2489,1044,1
2428,2023.8.31,건대입구,하차,1028,0,0,21.1,0.0,0,1,...,608,0,0,92,0.0,0,0,3619,2348,1
2429,2023.8.31,건대입구,승차,2296,0,0,21.1,0.0,0,1,...,608,1,0,92,0.0,0,0,1028,3361,1
2430,2023.8.31,건대입구,승차,2725,0,0,27.4,0.0,0,0,...,608,1,0,50,0.0,0,0,2296,2404,1


In [13]:
df = data
print(df.head())

# Summary of the data
print(df.describe())

# Columns to remove
cols_to_remove = ["Relative_Humidity", "cue", "Commute", "승하차", "Boarding_Alighting", "Transport_Date", "Station_Name"]

# Splitting the dataframe into four based on 'Commute' and '승하차' conditions
출근승차 = df[(df['Commute'] == 1) & (df['승하차'] == 1)].drop(columns=cols_to_remove)
출근하차 = df[(df['Commute'] == 1) & (df['승하차'] == 0)].drop(columns=cols_to_remove)
퇴근승차 = df[(df['Commute'] == 0) & (df['승하차'] == 1)].drop(columns=cols_to_remove)
퇴근하차 = df[(df['Commute'] == 0) & (df['승하차'] == 0)].drop(columns=cols_to_remove)

  Transport_Date Station_Name Boarding_Alighting  Passenger_Count  Typhoon  \
0       2022.1.1         건대입구                 승차              362        0   
1       2022.1.1         건대입구                 하차              180        0   
2       2022.1.1         건대입구                 승차             1606        0   
3       2022.1.1         건대입구                 하차             2758        0   
4       2022.1.2         건대입구                 승차              346        0   

   Heatwave  Temperature  Precipitation  Relative_Humidity  Commute  ...  \
0         0        -10.1            0.0                  0        1  ...   
1         0        -10.1            0.0                  0        1  ...   
2         0         -0.9            0.0                  0        0  ...   
3         0         -0.9            0.0                  0        0  ...   
4         0         -2.2            0.0                  0        1  ...   

   일월날짜  승하차  cue  습도   적설  파업  IsHoliday  이전 승하차수  일주일 전 승하차 수  잠실 스포츠  


In [14]:
# Checking for outliers in the 'Passenger_Count' column
# Using the IQR (Interquartile Range) method to detect outliers

Q1 = data['Passenger_Count'].quantile(0.25)
Q3 = data['Passenger_Count'].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as those outside of Q1 - 1.5 * IQR and Q3 + 1.5 * IQR
outliers = data[(data['Passenger_Count'] < (Q1 - 1.5 * IQR)) | (data['Passenger_Count'] > (Q3 + 1.5 * IQR))]

# Selecting the index and Transport_Date of the outliers
outlier_info = outliers[['Transport_Date', 'Passenger_Count']]
outlier_info.head() 

,Transport_Date,Passenger_Count


In [24]:
data=출근승차

In [44]:
len(data) - predict_ahead

596

In [57]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

all_predictions = []
window_size = 300
predict_ahead = 1
rmse = []  # To store root mean square errors

for i in range(window_size, len(data) - predict_ahead):
    train_data = data.iloc[:i]
    test_data = data.iloc[i:i + predict_ahead]

    X_train = train_data.drop('Passenger_Count', axis=1)
    y_train = train_data['Passenger_Count']
    X_test = test_data.drop('Passenger_Count', axis=1)
    y_test = test_data['Passenger_Count']

    # Fit the GLM model
    model = sm.GLM(y_train, X_train, family=sm.families.Gaussian())
    results = model.fit()

    # Make predictions
    predictions = results.predict(X_test)
    all_predictions.append(predictions.iloc[0])  # Assuming 'predictions' is a Series

# Convert list to Series
all_predictions_series = pd.Series(all_predictions)

In [63]:
from sklearn.svm import SVR

all_predictions = []
window_size = 300
predict_ahead = 1
rmse = []  # To store root mean square errors

for i in range(window_size, len(data) - predict_ahead):
    train_data = data.iloc[:i]
    test_data = data.iloc[i:i + predict_ahead]

    X_train = train_data.drop('Passenger_Count', axis=1)
    y_train = train_data['Passenger_Count']
    X_test = test_data.drop('Passenger_Count', axis=1)
    y_test = test_data['Passenger_Count']

    # Fit the SVM model
    model = SVR()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)
    all_predictions.append(predictions[0])  # Assuming 'predictions' is an array

    # Calculate RMSE for this prediction and store
    rmse.append(np.sqrt(np.mean((y_test - predictions) ** 2)))

# Convert list to Series
all_predictions_series = pd.Series(all_predictions)

# Calculate overall RMSE
overall_rmse = np.sqrt(np.mean(np.array(rmse)**2))
print(f"Overall RMSE: {overall_rmse}")


Overall RMSE: 826.6517559823989


In [64]:
# Define categorize_prediction function if not already defined
def categorize_prediction(value):
    if value >= 2500:
        return '위험'
    elif value >= 2100:
        return '매우혼잡'
    elif value >= 1800:
        return '혼잡'
    elif value >= 1000:
        return '보통'
    else:
        return '여유'

# Categorize all predictions
categories = all_predictions_series.apply(categorize_prediction)
print(categories.value_counts())


혼잡    296
dtype: int64


In [37]:
print(type(categories))
print(categories.head())

<class 'pandas.core.series.Series'>
2425    매우혼잡
dtype: object


In [38]:
# Assuming 'predictions' is a pandas Series containing all your prediction values
categories = predictions.apply(categorize_prediction)
print(categories.value_counts())


매우혼잡    1
dtype: int64


In [47]:
categories = categories.astype('category')


In [49]:
print(categories.dtypes)
print(categories.head())


category
0      혼잡
1    매우혼잡
2      혼잡
3    매우혼잡
4      혼잡
dtype: category
Categories (4, object): ['매우혼잡', '보통', '여유', '혼잡']


## 시각화

In [54]:
import matplotlib.pyplot as plt

# Count the occurrences of each category
category_counts = categories.value_counts()

# Create a bar plot
plt.bar(category_counts.index, category_counts.values)

# Add labels and title
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Prediction Categories')

# Show the plot
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\hyeseung\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hyeseung\AppData\Local\Temp\ipykernel_16688\3954584436.py", line 7, in <module>
    plt.bar(category_counts.index, category_counts.values)
  File "C:\Users\hyeseung\anaconda3\Lib\site-packages\matplotlib\pyplot.py", line 2439, in bar
    # Extract actual aspect ratio of array and make appropriately sized
       ^^^^^
  File "C:\Users\hyeseung\anaconda3\Lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    ## Plotting part 1: manually generated functions and wrappers ##
               ^^^^^
  File "C:\Users\hyeseung\anaconda3\Lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    num = next_num
           ^^^^^^^^
  File "C:\Users\hyeseung\anaconda3\Lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
    

## 재현율, 정밀도 

In [65]:
true_categories = data['Passenger_Count'].apply(categorize_prediction)


In [66]:
# Slice true_categories to match the length of categories
# Assuming you have categorized the true values in a similar manner
true_categories_aligned = true_categories[window_size:len(data) - predict_ahead]

# Now both true_categories_aligned and categories should have the same length
# Check the lengths to confirm
print(len(true_categories_aligned), len(categories))

# If the lengths match, proceed with calculating the confusion matrix
if len(true_categories_aligned) == len(categories):
    cm = confusion_matrix(true_categories_aligned, categories)
    precision = precision_score(true_categories_aligned, categories, average='weighted')
    recall = recall_score(true_categories_aligned, categories, average='weighted')

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
else:
    print("Length mismatch between actual and predicted categories.")


296 296
Precision: 0.013981464572680787
Recall: 0.11824324324324324


C:\Users\hyeseung\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


정밀도 (Precision): 약 0.8063입니다. 이는 모델이 '위험', '매우혼잡', '혼잡', '보통', '여유' 등의 범주를 예측할 때, 예측이 정확한 비율이 약 80.63%임을 의미합니다. 정밀도가 높을수록 거짓 양성(실제로는 해당 범주가 아닌데 해당 범주로 잘못 예측한 경우)의 수가 적다는 것을 나타냅니다.

재현율 (Recall): 약 0.7601입니다. 이는 실제로 각 범주에 속하는 데이터 중에서 모델이 얼마나 많은 비율을 정확하게 예측했는지를 나타내는 값입니다. 재현율이 높을수록 거짓 음성(실제로 해당 범주인데 다른 범주로 잘못 예측한 경우)의 수가 적다는 것을 의미합니다.